In [1]:
# Step 1: Create a table in the db to hold stock data.

import urllib.request as urlreq
import http.cookiejar as cookielib
import zipfile, os
import time
import sqlite3
from datetime import datetime

In [2]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

#create table
c.execute('CREATE TABLE prices (SYMBOL text, SERIES text, OPEN real, HIGH real, LOW real, CLOSE real, LAST real, PREVCLOSE real, TOTTRDQTY real, TOTTRDVAL real, TIMESTAMP date, TOTALTRADES real, ISIN text, PRIMARY KEY (SYMBOL, SERIES, TIMESTAMP))')
conn.commit()

In [3]:
# Step 2: Download stock files and unzip.

# Taken from zip processing lesson.
def download(localFilePath, remoteFileUrl):
    #header spoofing to bypass crawler blocks.
    hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive'}
    
    request = urlreq.Request(remoteFileUrl, headers=hdr)
    print('Attempting to fetch archive from ',remoteFileUrl)
    try:
        page = urlreq.urlopen(request)
        content = page.read() #Expecting this to be a zipfile.
        output = open(localFilePath,'wb')
        output.write(bytearray(content))
        output.close()
    except urlreq.HTTPError as e:
        print(e.fp.read())
        print('An HTTPError occured.')
        
# Also taken from zip processing lesson.
def unzip(localZipPath, localExtractPath):
    if os.path.exists(localZipPath):
        print('File ',localZipPath,' exists.')
        extractedFiles = []
        handler = open(localZipPath,'rb')
        zipHandler = zipfile.ZipFile(handler)
        for name in zipHandler.namelist():
            zipHandler.extract(name, localExtractPath)
            extractedFiles.append(localExtractPath+name)
            print('Extracted ',name,' from zip to ',localExtractPath+name,'.')
        print('Extracted ',str(len(extractedFiles)),' files in total.')
        handler.close()
        
def downloadAndUnzipForPeriod(listOfMonths, listOfYears):
    for year in listOfYears:
        for month in listOfMonths:
            for dayOfMonth in range(31):
                date = dayOfMonth + 1 #range(31) gives 0-30, needs a bump.
                dateStr = str(date)
                if date < 10:
                    dateStr = '0' + dateStr
                print(dateStr,'-',month,'-',year)
                fileName = 'cm'+str(dateStr)+str(month)+str(year)+'bhav.csv.zip'
                urlName = 'https://www.nseindia.com/content/historical/EQUITIES/'+year+'/'+month+'/'+fileName
                dirName = '/stockdata'
                if not os.path.isdir(dirName):
                    os.mkdir(dirName)
                localZipPath = dirName + '/' + fileName
                download(localZipPath, urlName)
                localExtractPath = localZipPath[:-4] #remove '.zip' from end
                unzip(localZipPath,localExtractPath)
                time.sleep(10) #pause to prevent website overload
    print('Files downloaded and extrated successfully')

In [4]:
# listOfMonths = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']
# listOfYears = ['2008','2009','2010','2011','2012','2013','2014','2015','2016']
# downloadAndUnzipForPeriod(listOfMonths,listOfYears)

# The preceeding code gets an HTTPError when attempting to fetch from the site, so data was downloaded manually to continue with the exercise.

01 - JAN - 2008
Attempting to fetch archive from  https://www.nseindia.com/content/historical/EQUITIES/2008/JAN/cm01JAN2008bhav.csv.zip
b'<HTML><HEAD>\n<TITLE>Access Denied</TITLE>\n</HEAD><BODY>\n<H1>Access Denied</H1>\n \nYou don\'t have permission to access "http&#58;&#47;&#47;www&#46;nseindia&#46;com&#47;content&#47;historical&#47;EQUITIES&#47;2008&#47;JAN&#47;cm01JAN2008bhav&#46;csv&#46;zip" on this server.<P>\nReference&#32;&#35;18&#46;a737d817&#46;1478363759&#46;c91f847\n</BODY>\n</HTML>\n'
An HTTPError occured.
02 - JAN - 2008
Attempting to fetch archive from  https://www.nseindia.com/content/historical/EQUITIES/2008/JAN/cm02JAN2008bhav.csv.zip
b'<HTML><HEAD>\n<TITLE>Access Denied</TITLE>\n</HEAD><BODY>\n<H1>Access Denied</H1>\n \nYou don\'t have permission to access "http&#58;&#47;&#47;www&#46;nseindia&#46;com&#47;content&#47;historical&#47;EQUITIES&#47;2008&#47;JAN&#47;cm02JAN2008bhav&#46;csv&#46;zip" on this server.<P>\nReference&#32;&#35;18&#46;a737d817&#46;1478363769&#46;c

KeyboardInterrupt: 

In [5]:
# Manually downloaded zip files are stored in stockdata. For ease of use, time to extract them.
filenames = os.listdir('stockdata')
for file in filenames:
    if file[-3:] != 'zip':
        continue
    file = 'stockdata/'+file
    print(file)
    unzip(file, file[:-4])

stockdata/cm01APR2008bhav.csv.zip
File  stockdata/cm01APR2008bhav.csv.zip  exists.
Extracted  cm01APR2008bhav.csv  from zip to  stockdata/cm01APR2008bhav.csvcm01APR2008bhav.csv .
Extracted  1  files in total.
stockdata/cm01APR2009bhav.csv.zip
File  stockdata/cm01APR2009bhav.csv.zip  exists.
Extracted  cm01APR2009bhav.csv  from zip to  stockdata/cm01APR2009bhav.csvcm01APR2009bhav.csv .
Extracted  1  files in total.
stockdata/cm01APR2010bhav.csv.zip
File  stockdata/cm01APR2010bhav.csv.zip  exists.
Extracted  cm01APR2010bhav.csv  from zip to  stockdata/cm01APR2010bhav.csvcm01APR2010bhav.csv .
Extracted  1  files in total.
stockdata/cm01APR2011bhav.csv.zip
File  stockdata/cm01APR2011bhav.csv.zip  exists.
Extracted  cm01APR2011bhav.csv  from zip to  stockdata/cm01APR2011bhav.csvcm01APR2011bhav.csv .
Extracted  1  files in total.
stockdata/cm01APR2013bhav.csv.zip
File  stockdata/cm01APR2013bhav.csv.zip  exists.
Extracted  cm01APR2013bhav.csv  from zip to  stockdata/cm01APR2013bhav.csvcm01APR

In [49]:
# Step 3: parse each file and insert the rows into the db

def insertRows(fileName,conn):
    # conn is an open connection to a db
    c = conn.cursor()
    lineNum = 0
    with open(fileName, 'r') as csvfile:
        # file uses double quotes as quotes
        lineReader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in lineReader:
            lineNum = lineNum + 1
            if lineNum == 1:
                print("Opened file, skipping header.")
                continue
            date_object = datetime.strptime(row[10], '%d-%b-%Y')
            rowTuple = [row[0], row[1], float(row[2]),float(row[3]),float(row[4]),float(row[5]),float(row[6]),float(row[7]),float(row[8]),float(row[9]),date_object, None,None]  
            c.execute('INSERT INTO prices VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)', rowTuple)
        conn.commit()
        print("Contents read, closing file.")   

In [50]:
import os, csv, datetime
from datetime import datetime

def datadump(parentDir, conn):
    # Data given stores each csv in its own folder (which for some reason, also ends in .csv).
    # To make my life easier, this will recursively search a parent directory (in this case, stockdata),
    # and push any csv's into the database (making the bold assumption they belong there).
    for file in os.listdir(parentDir):
        absDir = parentDir+'/'+file
        if not os.path.isfile(absDir):
            datadump(absDir, conn)
        elif file.endswith('.csv'):
            insertRows(absDir, conn)

In [51]:
localExtractFilePath = 'stockdata'
datadump(localExtractFilePath, conn)

Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing file.
Opened file, skipping header.
Contents read, closing fil

KeyboardInterrupt: 

In [52]:
# Step 4: Run a test query against the db to make sure it's been setup ok

tl = 'ICICIBANK'
series = 'EQ' #equity
c = conn.cursor()
cursor = c.execute('SELECT symbol, max(close), min(close), max(timestamp), min(timestamp), count(timestamp) FROM prices WHERE symbol = ? and series = ? GROUP BY symbol ORDER BY timestamp',(tl, series))
for row in cursor:
    print(row)

('ICICIBANK', 1794.1, 193.55, '2016-04-13 00:00:00', '2006-01-02 00:00:00', 1184)
